# II. Working with Dataframes

In this notebook, we'll start doing basic operations on dataframes. Specifically we'll look at

* Column operations
* Row operations
* Sorting
* Categorical variables

In [ ]:
using DataFrames, Dates, CSV

In [ ]:
# load csv file

df = CSV.read("fakedata.csv")

Let's take a preliminiary look at the data. The **first** and **last** functions can be used to examine the first  and last few lines of the dataframe.

In [ ]:
first(df,7)

In [ ]:
last(df,7)

You might want to get extract basic information about the dataframe such as the number of rows and columns in the data frame.  To get the dataframe size, you can use the **size** command which tells us there are 20 rows and 4 columns in the dataframe.

In [ ]:
nr, nc = size(df)

The **describe** function can be used to get summary statistics on the dataframe. The object returned is also a dataframe.

In [ ]:
describe(df)

We can restrict describe only to work on certain variables of interest and return only certain statistics.

In [ ]:
summstats = describe(df[:,[:v1, :v2]], :eltype, :min, :max, :median)

Since _summstats_ is itself a dataframe you can work with it as you would any other dataframe.

### Column operations:

We've seen the **size** function to get the dimensions of the dataframe. There is a **ncols** function if you just want the number of columns:

In [ ]:
nc = ncol(df)

Add a column using the [ ] notation and by providing a name for the column along with values using the assignment operator. Here we'll add a new column named _RandomStr_ to **df** that is just an array of random strings of length 10.

In [ ]:
using Random 
df[!,:RandStr] = [randstring(10) for j in 1:nr];

In [ ]:
df

You can do operations on existing columns in the dataframe to create new columns. We know that **v4** is a Date type so we can use Julia's _Dates_ package to work with these types.

In [ ]:
eltype(df.v4)

Let's add a new column that provides the day for each date value in **v4**.

In [ ]:
df[!, :v4_day] =  day.(df.v4);

In [ ]:
df

We can create a new column variable calculated based on other columns in the dataframe and insert this into a specific location in the dataframe using **insertcols!**. This is an in place operation.

The first argument to insertcols! is the name of the data frame, the second argument is the column index number where you want the new column to be placed, and the last argument specifies the data values of the new column.

In [ ]:
insertcols!(df, 6, elapsed_days = df.v5 - df.v4)

Note the type of df.elapsed. It's an array with element of type **Day** (not **Int64**). **Day** is a built-in type that is part of the _Dates_ package.

In [ ]:
typeof(df.elapsed_days)

You can also create a new column by mapping an arbitrary function to another column. Here let's use Julia's __map__ function to create new column called <i>elapsed_hours</i> equal to <i>elapsed_days</i> converted into hours.

In [ ]:
df[!, :elapsed_hours] = map(t -> Dates.Hour(t), df[:, :elapsed_days]); 
# above is the same as df[!, :elapsed_hours] =  Dates.Hour.(df.elapsed_days)

In [ ]:
df

As you likely noticed, not all columns were displayed. You can use the __show__ command to see all column variables for a few rows.

__By default Jupyter Notebook will limit the number of rows and columns when displaying a data frame to roughly fit the screen size.__

In [ ]:
show(df, allcols=true)

If you want to drop a column you can use **select!**. This is an in place operation and therefore will modify the dataset. Use **select** if you want to do the non in place version of this operation.

We'll drop the <i>elapsed_hours</i> and _v5_ columns.

In [ ]:
select!(df, Not([:elapsed_hours, :v5]));

In [ ]:
names(df)

The __select!__ command can also be used to keep a specific subset of columns:

In [ ]:
select!(df, [:v1, :v2, :v3, :v4, :elapsed_days, :RandStr, :v4_day])

To change the name of a column variable you can use the __rename!__ function. We'll change the column name of <i>elapsed_days</i> to be <i>elap_days</i> and _v3_ to be _Category_. The __rename__ function is the non in place version of this operation.

In [ ]:
rename!(df, :elapsed_days => :elap_days, :v3 => :Category,);

In [ ]:
names(df)

A simple way to reorder the columns is by indexing. In this case you index using the column names and listing them in the desired order. Here we make _Category_ the first variable in the dataframe.

In [ ]:
df[:, [3, 1, 2, 4, 5, 6, 7]]

You can do the same thing in place using the **permutecols!** function.

The last function we will look at in regard to working with columns is **eachcol**. This is a function that allows you to iterate over the columns of your dataframe: 

`eachcol(df, bool)`

The first argument to **eachcol** is the name of the dataframe whose columns you want to iterate over and the second is a boolean. If you specify true for the boolean argument then for each column you get a __Pair__ whose first element is the column name (symbol) and second element is the corresponding column values.

We can **eachcol** is to pick out columns (based on their type) and perform an operation on that column. Here we calculate basic summary statistics **only** for columns in the **df** dataframe that are subtypes of _Real_.

In [ ]:
using Statistics

for col in eachcol(df, true)
    if (eltype(col[2]) <: Real) 
        println(col[1], "\n  ", "Mean: ", round(mean(col[2]), digits=3), 
            "\n  Max: ", round(maximum(col[2]), digits=1),
            "\n  Min: ", round(minimum(col[2]), digits=1),
            "\n  Range: ", round(maximum(col[2]) - minimum(col[2]), digits=1)
        )
    end
end

### Row operations:

In [ ]:
df = CSV.read("fakedata.csv")

If you want the number of rows in your dataframe there is a __nrows__ function for that:

In [ ]:
nr = nrow(df)

When indexing into a dataframe the first index indicates the rows and the second the columns. **Indexing a dataframe works as you expect**. The colon indicates all elements or items along the dimension.

For example, if we wanted to "subset" all rows and all columns:

In [ ]:
df[:, :]

If we only wanted rows with a certain index we would specify that in the first argument. Here we get rows 5 through 9.

In [ ]:
df[5:9, :]

When reading in data using CSV.read, by default, the columns in the resulting dataframe are immutable so that individual elements can not be modified.

In [ ]:
df[2, :v1] = 4

In [ ]:
typeof(df.v1)

In [ ]:
isimmutable(df.v1)

Following the suggestion of the error message we can pass in the copycols argument to **CSV.read**, setting it equal to true, which will result in a dataframe that allows us to change individual elements or slices in the dataframe:

`df = CSV.read("fakedata.csv", copycols = true)`

Another option is to make a copy of the data via the **copy** command:

In [ ]:
df = copy(df)

In [ ]:
typeof(df.v1)

In [ ]:
isimmutable(df.v1)

In [ ]:
df[2, :v1]

In [ ]:
df[2, :v1] = 4

You can update slices as well as long as the types and dimensions match.

In [ ]:
first(df, 10)

In [ ]:
df[1:5, [:v1, :v2]] = [[1, 2, 3, 6, 5] randn(5,1)]

In [ ]:
df

Suppose we wanted to subset based on the value of a column variable? For example, if we wanted all rows of data where _v3_ has the value "a":

In [ ]:
df[df.v3 .== "a", :]

And if we wanted the rows of data where _v3_ has value "a" and _v2_ is greater than 0 and just columns _v1, v2,_ and _v3_?

In [ ]:
df[(df.v3 .== "a") .& (df.v2 .> 0), [:v1, :v2, :v3]]

You can also use Julia's built-in __filter__ function to subset data. You can use either __filter__ or __filter!__. The former will return a copy of the dataframe with rows that satisfy the filter; the latter will actually modify the dataframe in place __keeping__ only rows that satisfy the filter.

The first argument will be the filter itself almost always expressed as an anonymous function and the second argument will just be the name of the dataframe the filter should be applied to.

Here we filter on rows where the vale of _v1_ is greather than 1.

In [ ]:
filter(row -> row[:v1] > 1, df)

If you need to delete rows you can use the __deleterows!__ function and provide the row ids to delete.

In [ ]:
deleterows!(df, [3, 15])

A more useful thing might be to get rid of duplicate rows. The first occurrence of the duplicate row is kept in the dataframe while all others are regarded as duplicates and removed.

To this end you can use either the **unique** or **unique!** functions. The former returns a copy of the dataframe with deleted duplicate rows while the latter is an in place operation.

`unique(df, cols)` <br/>
`unique!(df, cols)`

The **cols** argument is optional. If you specify the **cols** argument the comparison will be made using only the specified columns. If you don't specify the **cols** argument then the comparison will be made using all columns which means only rows with identical values across all columns will be matches.

We can see in our dataframe above that when only considering columns _v1_ and _v3_ there are duplicate rows. For example there are many rows with _v1_ equal to 1 and _v3_ equal to "a" or with _v1_ equal to 2 and _v3_ equal to a.

To get rid of duplicate rows on columns v1 and v3:

In [ ]:
unique(df, [:v1, :v3])

The **nonunique** function will tell you if a given row is a duplicate of any row **before** it.

In [ ]:
nonunique(df, [:v1, :v3])

Lastly if you need to randomly reorder the rows of data you can use the __shuffle__ function.

In [ ]:
using Random

df[shuffle(1:nrow(df)), :]

### Sorting:

Sorting allows you to order the data in different ways. Typically, you'll want to order the data with respect to a variable in the dataframe. The functions you can use for sorting are __sort__ or __sort!__. The latter is the in place version of the former.

In [ ]:
df = DataFrame(A = [0, 0, 1, 1, 0, 0, 1], B = [0, 1, 0, 0, 1, 0, 1], 
               C = [0, 1, 1, 1, 0, 1, 1], D = [1, 2, 3, 4, 2, 1, 0])

To check if a dataframe is sorted by columns you can use the __issorted__ function:

In [ ]:
issorted(df)

In [ ]:
issorted(df, :B) #checks if dataframe is sorted by column B

Simply calling sort on your dataframe will sort the dataframe using __all__ columns of data. Specifically, it will sort by the first column, then by the second column, then by the third column, etc. In this case it sorts by _A, B, C,_ and then _D_. By default the sorting is done in ascending order.

In [ ]:
sort(df)

To sort by specified columns you can pass the column names as parameters to the sort function. For example, here we just sort by column _C_.

In [ ]:
sort(df, :C)

If we wanted order first by column <i>C</i> and then by column <i>A</i> you would specify both variables in an array.

In [ ]:
sort(df, [:C, :A])

As mentioned above, sorting by default is done in ascending order. However, you can change this via the __rev__ keyword argument. For example, if we wanted to sort **df** by column _D_ in descending order we would set rev to true.

In [ ]:
sort(df, :D, rev=true)

If sorting by two or more column variables you can specify how each column should be sorted, i.e. ascending or descending. Here we'll sort by the variables D and C, but D will be sorted in descending order and C in ascending order.

In [ ]:
sort(df, (:D, :C), rev = (true, false))

### Categorical variables:

In [ ]:
df = DataFrame(A = [0, 0, 1, 1, 0, 0, 1], B = [0, 1, 0, 0, 1, 0, 1], 
               C = [0, 1, 1, 1, 0, 1, 1], D = [1, 2, 3, 4, 2, 1, 0])

Some of the variables in a dataframe may take on values that represent categories, i.e. categorical, ordinal, etc. variables.

Let's add a categorial variable called _t_ to the **df** dataframe.

In [ ]:
df.t = ["High", "High", "High", "Low", "Low", "Medium", "Medium"];

As you can see the _t_ variable is an array with element type **String**. So there is no notion of this variable having levels or being categorical.

In [ ]:
typeof(df.t)

In Julia you can work with categorical types via the __CategoricalArrays__ package.

In [ ]:
using CategoricalArrays

If you have an existing dataframe and want to convert a variable to be categorical you can use the **categorical!** function passing the name of the dataframe and variable to be converted. The non in place version of this operation is **categorical**.

In [ ]:
categorical!(df, :t)

In [ ]:
typeof(df.t)

You can see what the levels of a categorical variables are:

In [ ]:
levels(df.t)

We can also verify that this categorical variable has no ordering:

In [ ]:
isordered(df.t)

If we want we can impose an ordering on the variable _t_, e.g. Low < Medium < High. The default ordering is based on the current output of the levels command, i.e. the ordering will be High < Low < Medium. If we want the former we can chave the order using the **levels!** function. The first argument is the categorical variable and the second is the desired ordering.

In [ ]:
levels!(df.t, ["Low", "Medium", "High"]);

In [ ]:
levels(df.t)

Now we can use the **ordered!** function to impose an ordering on the _t_ column variable. The result of the **ordered!** function is based on the output of the **levels** command.

In [ ]:
ordered!(df.t, true);

In [ ]:
isordered(df.t)

Now with respect to the _t_ column variable, Low < Medium < High.

In [ ]:
df.t[1]

In [ ]:
df.t[4]

In [ ]:
df.t[1] < df.t[4]

In this lesson we covered:
* Column operations such as creating new columns, selecting columns, renaming columns, etc.
* Row operations including subsetting rows, filtering, and identifying duplicate rows.
* Sorting dataframes.
* Categorical data.